Kmeans

In [2]:
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import sys

from sklearn import preprocessing

In [30]:
folder_path = '../datasets/ml-25m'
# rating_df = pd.read_csv(os.path.join(folder_path, "ratings.csv"))
rating_df = pd.read_parquet(os.path.join(folder_path, "ratings_sampled.parquet"))
print(rating_df.shape)
rating_df.head()

(250001, 4)


,userId,movieId,rating,timestamp
index,,,,
6101963,39561,491,3.0,839631242
12940169,83787,3082,2.5,1206219958
9527588,61879,7669,4.5,1345706259
979862,6568,1073,3.0,858100721
10329818,66999,94864,3.5,1449105450


In [48]:
rt_mat =rating_df.pivot(index="userId", columns="movieId", values="rating").values

In [49]:
rt_mat.shape

(90135, 14249)

In [43]:
rating_mt = rating_df[["userId","movieId","rating"]].pivot(index="userId", columns="movieId", values="rating").values

In [44]:
rating_mt.shape

(90135, 14249)

In [45]:
rating_mt

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [31]:
movie_df = pd.read_csv(os.path.join(folder_path, "movies.csv"))
print(movie_df.shape)
movie_df.head()

(62423, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [38]:
pd.merge(rating_df, movie_df, how='left', on='movieId')

,userId,movieId,rating,timestamp,title,genres
0,39561,491,3.0,839631242,"Man Without a Face, The (1993)",Drama
1,83787,3082,2.5,1206219958,"World Is Not Enough, The (1999)",Action|Adventure|Thriller
2,61879,7669,4.5,1345706259,Pride and Prejudice (1995),Drama|Romance
3,6568,1073,3.0,858100721,Willy Wonka & the Chocolate Factory (1971),Children|Comedy|Fantasy|Musical
4,66999,94864,3.5,1449105450,Prometheus (2012),Action|Horror|Sci-Fi|IMAX
...,...,...,...,...,...,...
249996,139926,761,4.0,892217941,"Phantom, The (1996)",Action|Adventure
249997,31881,26662,4.5,1500453256,Kiki's Delivery Service (Majo no takkyûbin) (1...,Adventure|Animation|Children|Drama|Fantasy
249998,125696,904,4.5,1374263491,Rear Window (1954),Mystery|Thriller
249999,29824,1721,4.0,883680104,Titanic (1997),Drama|Romance


In [32]:
tag_df = pd.read_csv(os.path.join(folder_path, "genome-tags.csv"))
tag_score_df = pd.read_csv(os.path.join(folder_path, "genome-scores.csv"))
links_df = pd.read_csv(os.path.join(folder_path, "links.csv"))

In [33]:
tag_df.head(3)

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century


In [34]:
tag_score_df.head(3)

,movieId,tagId,relevance
0,1,1,0.02875
1,1,2,0.02375
2,1,3,0.06250


In [35]:
links_df.head(3)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0


In [ ]:
dtypes_to_check = ["string", "float", "int"]
customer_df

In [16]:
customer_df.select_dtypes("object")

,customer_id,club_member_status,fashion_news_frequency,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,ACTIVE,NONE,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,ACTIVE,NONE,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,ACTIVE,NONE,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,ACTIVE,NONE,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,ACTIVE,Regularly,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...
...,...,...,...,...
1371975,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,ACTIVE,NONE,7aa399f7e669990daba2d92c577b52237380662f36480b...
1371976,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,ACTIVE,NONE,3f47f1279beb72215f4de557d950e0bfa73789d24acb5e...
1371977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,ACTIVE,Regularly,4563fc79215672cd6a863f2b4bf56b8f898f2d96ed590e...
1371978,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,ACTIVE,Regularly,8892c18e9bc3dca6aa4000cb8094fc4b51ee8db2ed14d7...


In [14]:
customer_df.dtypes

customer_id                object
FN                        float64
Active                    float64
club_member_status         object
fashion_news_frequency     object
age                       float64
postal_code                object
dtype: object

In [3]:
# TODO: Need to make this process dynamic, given dataset we need to figure out its dtypes then
# apply appropriate preprocessing steps.

df.dtypes

age         int64
country    object
dtype: object

In [8]:
# TODO: should use correct numpy array types for maximum efficieny
df.to_numpy()

array([[22, 'Korea'],
       [21, 'Canada'],
       [23, 'Canada'],
       [33, 'Korea'],
       [35, 'Korea'],
       [33, 'Korea'],
       [33, 'Canada'],
       [55, 'Singapore']], dtype=object)

In [6]:
df.select_dtypes("int")

,age
0,22
1,21
2,23
3,33
4,35
5,33
6,33
7,55


In [4]:
df.describe()

,age
count,8.000000
mean,31.875000
std,10.973182
min,21.000000
25%,22.750000
50%,33.000000
75%,33.500000
max,55.000000


In [3]:
le = preprocessing.LabelEncoder()
df['country'] = le.fit_transform(df['country'])
# df = df.reset_index().rename(columns={"index":"user_id"})

In [4]:
country_le_map = dict(zip(le.classes_, le.transform(le.classes_)))

In [5]:
df.head()

,age,country
0,22,1
1,21,0
2,23,0
3,33,1
4,35,1


In [20]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x = df['age'],
        y = df['country'],
        mode='markers'
    )
)

fig.show()

In [17]:
"""
K-means clustering implementation

1. choose random k-points
2. calculate distance from k-point and all other values.

"""

In [13]:
from sklearn.cluster import KMeans
X = df.values
kmeans = KMeans(n_clusters=3, n_init=10)
kmeans.fit(X)
df['cluster'] = kmeans.predict(X)

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x = df['age'],
        y = df['country'],
        mode='markers',
        marker=dict(color=df['cluster'])
    )
)

fig.show()

In [15]:
df

,age,country,cluster
0,22,1,1
1,21,0,1
2,23,0,1
3,33,1,0
4,35,1,0
5,33,1,0
6,33,0,0
7,55,2,2
